# Add No-Fly Zones to Area Data

We now have a separate GeoDataFrame containing all linear infrastructures and natural corridors for the selected area or city, and another GeoDataFrame containing all no-fly zones in the Netherlands. This notebook ensures that the relevant no-fly zones are added to the area-specific data, resulting in a ready-to-use GeoDataFrame for generating a graph that avoids paths through no-fly zones.

In [ ]:
# Import necessary libraries
import geopandas as gpd
import pandas as pd
import osmnx as ox

In [ ]:
# Define the city or area for which we want to assign no-fly zones

# city = 'borsele'
# city = 'breda'
city = 'alphen-waddinxveen'

In [ ]:
# Define the boundaries for the area of interest, use the same boundaries as used in the area data

# boundaries = ['Borsele, Zeeland, Netherlands']
# boundaries = ['Breda, North Brabant, Netherlands']
boundaries = [
    'Alphen aan den Rijn, Zuid-Holland, Netherlands',
    'Waddinxveen, Zuid-Holland, Netherlands',
    'Boskoop, Zuid-Holland, Netherlands'
] 

In [ ]:
osm_data = gpd.read_file(f"../2.risk_analysis/output/{city}/osm_data_with_risk_{city}.geojson")

In [ ]:
no_fly_zones = gpd.read_file("../3.no_fly_zones/input/gdf_no_fly_zones_nl_27feb.geojson")

In [ ]:
# Get the boundaries as a GeoDataFrame
boundary = ox.geocode_to_gdf(boundaries)

# Make sure both are in the same CRS
no_fly_zones = no_fly_zones.to_crs(boundary.crs)

# Filter no-fly zones that are within the area
no_fly_in_boundarie = no_fly_zones[no_fly_zones.geometry.intersects(boundary.geometry.iloc[0])].copy()

In [ ]:
no_fly_in_boundarie['risk'] = 'no_fly_zone'

In [ ]:
# Concatenate the OSM data with the no-fly zones within the boundaries
concatenated_gdf = pd.concat([osm_data, no_fly_in_boundarie], ignore_index=True)

In [ ]:
mask = concatenated_gdf['risk'] == 'no_fly_zone'
subset = concatenated_gdf[mask]

counts = subset.groupby(subset.geometry.type)['area_type'].value_counts()

print("Aantal area_types per geometry type voor risk='no_fly_zone':")
print(counts)

In [ ]:
concatenated_gdf.to_file(f"../3.no_fly_zones/output/data_for_graph_{city}.geojson", driver="GeoJSON")